In [1]:
from torch.utils.data import DataLoader

from src.concept_bottleneck.dataset import CUB200ImageToClass

batch_size = 16
num_workers = 2

training_data = CUB200ImageToClass(train=True)
test_data = CUB200ImageToClass(train=False)

training_dataloader = DataLoader(training_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)


In [2]:
import torch

from src.concept_bottleneck.networks import get_mlp

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = get_mlp().to(device)


Using device: cuda


In [3]:
import numpy as np


def train(
    model: torch.nn.Module,
    dataloader: DataLoader[tuple[torch.Tensor, np.int_]],
    trained_image_to_attributes_model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: str,
):
    model.train()
    trained_image_to_attributes_model.eval()
    size = len(dataloader.dataset)  # type: ignore
    for batch, (x, y) in enumerate(dataloader):
        x = trained_image_to_attributes_model(x.to(device))
        y = y.to(device)

        logits = model(torch.sigmoid(x))
        loss = loss_fn(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            print(f"loss: {loss.item():>7f} [{batch * len(x):>5d}/{size:>5d}]")


def test(
    model: torch.nn.Module,
    dataloader: DataLoader[tuple[torch.Tensor, np.int_]],
    trained_image_to_attributes_model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    device: str,
):
    model.eval()
    trained_image_to_attributes_model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = trained_image_to_attributes_model(x.to(device))
            y = y.to(device)

            logits = model(torch.sigmoid(x))
            test_loss += loss_fn(logits, y).item()
            correct += (logits.argmax(dim=1) == y).sum().item()

    test_loss /= len(dataloader)
    accuracy = correct / len(dataloader.dataset)  # type: ignore

    return test_loss, accuracy


In [4]:
from src.concept_bottleneck.train import TrainFn, TestFn, run_epochs, MODEL_PATH
from src.concept_bottleneck.inference import (
    load_image_to_attributes_model,
    INDEPENDENT_IMAGE_TO_ATTRIBUTES_MODEL_NAME,
    SEQUENTIAL_ATTRIBUTES_TO_CLASS_MODEL_NAME,
)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

trained_image_to_attributes_model = load_image_to_attributes_model(
    INDEPENDENT_IMAGE_TO_ATTRIBUTES_MODEL_NAME, device
)

train_fn: TrainFn = lambda model: train(
    model,
    training_dataloader,
    trained_image_to_attributes_model,
    loss_fn,
    optimizer,
    device,
)
test_fn: TestFn = lambda model, dataloader: test(
    model, dataloader, trained_image_to_attributes_model, loss_fn, device
)

epochs = 500


def on_better_accuracy(model: torch.nn.Module, accuracy: float):
    print(
        f"Saving model to {SEQUENTIAL_ATTRIBUTES_TO_CLASS_MODEL_NAME} with accuracy {100 * accuracy:>0.4f}%"
    )
    torch.save(
        model.state_dict(), MODEL_PATH / SEQUENTIAL_ATTRIBUTES_TO_CLASS_MODEL_NAME
    )


run_epochs(
    epochs,
    model,
    train_fn,
    test_fn,
    training_dataloader,
    test_dataloader,
    on_better_accuracy,
)


Using cache found in /home/shuangwu/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 1/500-------------------
loss: 5.357333 [    0/ 5994]
loss: 4.265321 [ 1600/ 5994]
loss: 3.860470 [ 3200/ 5994]
loss: 3.470402 [ 4800/ 5994]
Training Loss: 3.1721, Training Accuracy: 30.7307%
Test Loss: 3.2647, Test Accuracy: 28.5640%
Saving model to sequential_attributes_to_class.pth with accuracy 28.5640%
Epoch 2/500-------------------
loss: 3.251681 [    0/ 5994]
loss: 3.175524 [ 1600/ 5994]
loss: 2.910761 [ 3200/ 5994]
loss: 2.831971 [ 4800/ 5994]
Training Loss: 2.5456, Training Accuracy: 39.7564%
Test Loss: 2.6765, Test Accuracy: 37.5043%
Saving model to sequential_attributes_to_class.pth with accuracy 37.5043%
Epoch 3/500-------------------
loss: 2.357429 [    0/ 5994]
loss: 2.444016 [ 1600/ 5994]
loss: 2.105848 [ 3200/ 5994]
loss: 2.401489 [ 4800/ 5994]
Training Loss: 2.2636, Training Accuracy: 44.7447%
Test Loss: 2.4183, Test Accuracy: 41.9054%
Saving model to sequential_attributes_to_class.pth with accuracy 41.9054%
Epoch 4/500-------------------
loss: 2.380240 [    0/ 5

OrderedDict([('0.weight',
              tensor([[-0.3324, -2.8829,  0.4667,  ..., -1.2041, -0.8012, -1.7086],
                      [-0.3720, -1.6699, -0.2044,  ...,  0.4357, -0.3627, -2.8148],
                      [-0.5330, -2.3994, -0.4541,  ..., -1.2451, -0.5242, -1.8711],
                      ...,
                      [-0.1129,  3.2309, -0.0998,  ...,  2.2661,  2.1056, -4.3045],
                      [ 1.1753,  0.6442,  0.1710,  ...,  3.1050,  1.2644, -2.9183],
                      [-0.1710,  0.3591, -0.1896,  ..., -0.2040, -2.5275,  2.2067]],
                     device='cuda:0')),
             ('0.bias',
              tensor([-1.2289e-02,  1.3305e+00, -2.5209e+00,  1.4965e+00,  1.5482e+00,
                       1.7857e+00,  3.9253e-01,  1.5506e+00,  3.3860e+00, -2.5045e-02,
                       1.0220e+00,  5.2778e-01,  6.9424e-01,  7.0044e-01, -8.3225e-01,
                       8.3236e-01,  6.7667e-01,  2.8322e+00, -2.1689e+00, -2.3142e+00,
                      -1.1426e